In [33]:
import random
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
import os

In [34]:
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [35]:
print(type(device))

<class 'torch.device'>


EPOCHS: 전체 train dataset에 대하여 얼마나 training 할 것인가 <br>
LR(learning rate): step size라고도 하며 gradient descent 할 때 얼마나 변경할 것인가 <br>
BS(batch size): batch 1개에 dataset의 개수<br>
SEED: random 값을 고정

In [36]:
EPOCHS = 1000 
LR = 1e-2
BS = 10000
SEED = 41

In [37]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED) # Seed 고정

In [38]:
train_df = pd.read_csv('./data/train.csv')
train_df = train_df.drop(columns=['ID'])
val_df = pd.read_csv('./data/val.csv')
val_df = val_df.drop(columns=['ID'])

In [39]:
class MyDataset(Dataset):
    def __init__(self, df, eval_mode):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
        
    def __len__(self):
        return len(self.df)

In [40]:
train_dataset = MyDataset(df=train_df, eval_mode=False)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=2)

val_dataset = MyDataset(df = val_df, eval_mode=True)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=2)

# Chage hyperparameters in Neural Network
1. LeakyReLU -> GELU </br>
    최고 macro f1 score까지 더 빨리 왔지만 더이상 올라가지 않음 <br>
    suppose: 아마 오답에 매우 가까운 정답이 있거나 정답에 매우 가까운 오답이 있음, 전자일 확률이 높음<br>
    A: 정답에 매우 가까운 오답들이 너무 많음....(feat. EDA) 
    
2. Linear layer를 더 늘려보자 =>  macro f1-score가 올라가다가 다시 내려감 결국 50으로...
3. threshold를 더 늘려보자. 95% -> 98% => 85% 정도에서 고정..
4. 원래 방식대로 차원을 줄이고 나서 늘려볼까? => 성능 더 안 좋음..
     

In [41]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.Encoder = nn.Sequential(
            nn.Linear(30,64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Linear(64,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
        )
        self.Decoder = nn.Sequential(
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Linear(64,30),
        )
        
    def forward(self, x):
        x = self.Encoder(x)
        x = self.Decoder(x)
        return x

In [42]:
from utils.vector_sim import TS_SS

In [43]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        # Loss Function
        self.criterion = nn.L1Loss().to(self.device)
        
    def fit(self, ):
        self.model.to(self.device)
        best_score = 0
        for epoch in range(EPOCHS):
            self.model.train()
            train_loss = []
            for x in iter(self.train_loader):
                x = x.float().to(self.device)
                self.optimizer.zero_grad()

                _x = self.model(x)
                loss = self.criterion(x, _x)

                loss.backward()
                self.optimizer.step()

                train_loss.append(loss.item())

            score = self.validation(self.model, 10)
            print(f'Epoch : [{epoch}] Train loss : [{np.mean(train_loss)}] Val Score : [{score}])')

            if self.scheduler is not None:
                self.scheduler.step(score)

            if best_score < score:
                best_score = score
                torch.save(model.module.state_dict(), './checkpoint/best_model.pth', _use_new_zipfile_serialization=False)
    
    def validation(self, eval_model, thr):
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred = []
        true = []
        with torch.no_grad():
            for x, y in iter(self.val_loader):
                # x: (batch size, 30)
                x = x.float().to(self.device)

                _x = self.model(x)
                
                similarity = TS_SS(device=device)
                diff = similarity(x, _x).cpu().tolist()
                print(f"Min: {min(diff)} / Max: {max(diff)}")
                batch_pred = np.where(np.array(diff)>thr, 1,0).tolist()
                pred += batch_pred
                true += y.tolist()

        return f1_score(true, pred, average='macro')

In [44]:
model = nn.DataParallel(AutoEncoder())
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)

trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()

Min: 15.780115127563477 / Max: 106581.9296875
Min: 13.719902038574219 / Max: 32662.65234375
Min: 10.676398277282715 / Max: 29531.91015625
Epoch : [0] Train loss : [0.46706242362658185] Val Score : [0.0010529271374420891])
Min: 13.68333625793457 / Max: 47576.1640625
Min: 12.625732421875 / Max: 53288.546875
Min: 12.893082618713379 / Max: 33076.66015625
Epoch : [1] Train loss : [0.2670592355231444] Val Score : [0.0010529271374420891])
Min: 8.459122657775879 / Max: 29979.78515625
Min: 8.144327163696289 / Max: 39202.8359375
Min: 7.951747894287109 / Max: 18465.51953125
Epoch : [2] Train loss : [0.19676533589760462] Val Score : [0.005120563395392736])
Min: 3.687180519104004 / Max: 18664.390625
Min: 3.610445737838745 / Max: 25044.474609375
Min: 3.191511631011963 / Max: 7777.49609375
Epoch : [3] Train loss : [0.1644363341232141] Val Score : [0.1929627928639546])
Min: 2.308039903640747 / Max: 14302.421875
Min: 2.2992610931396484 / Max: 18027.994140625
Min: 1.9727038145065308 / Max: 5471.97216796

Exception ignored in: <function _releaseLock at 0x7f619f261f70>
Traceback (most recent call last):
  File "/home/common/miniconda3/envs/jhoon/lib/python3.8/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


Min: 0.84932941198349 / Max: 7831.13623046875
Min: 0.7066828012466431 / Max: 7507.498046875
Min: 0.47122296690940857 / Max: 1832.081298828125
Epoch : [13] Train loss : [0.09588159434497356] Val Score : [0.4821448936534185])
Min: 0.8365642428398132 / Max: 7470.1689453125
Min: 0.6924500465393066 / Max: 6768.69482421875
Min: 0.41346192359924316 / Max: 1681.590576171875
Epoch : [14] Train loss : [0.09386853128671646] Val Score : [0.4840388096053177])
Min: 0.8010923862457275 / Max: 7335.392578125
Min: 0.6426610350608826 / Max: 6470.04833984375
Min: 0.4902832508087158 / Max: 1616.86181640625
Epoch : [15] Train loss : [0.091129786024491] Val Score : [0.48548093400736514])
Min: 0.8794159889221191 / Max: 7764.587890625
Min: 0.6259289383888245 / Max: 6812.66015625
Min: 0.5260598063468933 / Max: 1838.993896484375
Epoch : [16] Train loss : [0.0910023699204127] Val Score : [0.48352893073288505])
Min: 0.886025071144104 / Max: 7309.12548828125
Min: 0.6557742953300476 / Max: 6348.638671875
Min: 0.4655

In [ ]:
model = AutoEncoder()
model.load_state_dict(torch.load('./checkpoint/best_model.pth'))
model = nn.DataParallel(model)
model.eval()

DataParallel(
  (module): AutoEncoder(
    (Encoder): Sequential(
      (0): Linear(in_features=30, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Linear(in_features=64, out_features=128, bias=True)
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.01)
    )
    (Decoder): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Linear(in_features=64, out_features=30, bias=True)
    )
  )
)